In [167]:
import numpy as np
from scipy.spatial.distance import pdist, cdist, squareform
from math import sqrt
from numba import njit,jit

In [403]:
@njit
def mydist(XA,XB,mode= 'euclidean'):
    if mode == 'euclidean':
        aux3 = []
        for i in range (len(XA)):
            aux2 = []
            for j in range (len(XB)):
                aux1 = []
                bux1 = 0 
                # calculate distance
                for e1, e2 in zip(XB[j],XA[i,:]):
                    aux1.append((e1-e2)**2)
                    bux1 += ((e1-e2)**2)

                    #aux1.append(sqrt(sum((e1-e2)**2)))
                aux2.append(bux1**(0.5))
            aux3.append(aux2)
        return aux3
    
    if mode == 'cosine':
        aux2 = []
        for i in range(len(XA)):
            aux1 = []
            for j in range(len(XB)):
                A = Uniquesample[j,:] - 1
                B = Uniquesample[i,]
                dot = 0
                denom_a = 0
                denom_b = 0
                for k in range (len(XB[j,:])):
                    dot += (XB[j,k]*XA[i,k])
                    denom_a += (XB[j,k] * XB[j,k])
                    denom_b += (XA[i,k] * XA[i,k])

                aux1.append(1 - ((dot / ((denom_a ** 0.5) * (denom_b ** 0.5)))))
            aux2.append(aux1)

        return aux2

In [404]:
Uniquesample = np.array(([1,2,3],[4,4.5,5],[7,8,9]))
print(Uniquesample)

[[1.  2.  3. ]
 [4.  4.5 5. ]
 [7.  8.  9. ]]


In [405]:
%%time
eucli1 = cdist(Uniquesample,Uniquesample[0:2,:]-1)
print(uspi)

[[0.0438171125324851, 0.03662466183423507], [0.17142857142857137, 0.00032877613973902875], [0.16519031121168382, 0.00010379006756999765]]
CPU times: user 0 ns, sys: 2.47 ms, total: 2.47 ms
Wall time: 2.7 ms


In [406]:
%%time
eucli2 = mydist(Uniquesample,Uniquesample[0:2,:]-1)
print(np.array(uspi))

[[4.38171125e-02 3.66246618e-02]
 [1.71428571e-01 3.28776140e-04]
 [1.65190311e-01 1.03790068e-04]]
CPU times: user 791 ms, sys: 1.73 ms, total: 793 ms
Wall time: 794 ms


In [407]:
cos1 = cdist(Uniquesample,Uniquesample[0:2,:]-1,metric = 'cosine')
print(np.array(uspi))

[[4.38171125e-02 3.66246618e-02]
 [1.71428571e-01 3.28776140e-04]
 [1.65190311e-01 1.03790068e-04]]


In [408]:
uspi = mydist(Uniquesample,Uniquesample[0:2,:]-1,mode = 'cosine')
print(np.array(uspi))

[[4.38171125e-02 3.66246618e-02]
 [1.71428571e-01 3.28776140e-04]
 [1.65190311e-01 1.03790068e-04]]
